<a href="https://colab.research.google.com/github/tottenjordan/artofficialintelligence/blob/master/Aylien_news_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pip install aylien news api

In [2]:
!pip install aylien_news_api
import time
import datetime
import csv
import aylien_news_api
from aylien_news_api.rest import ApiException

    100% |████████████████████████████████| 61kB 2.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/aa/31/3c/e2b4bd35b18bb499d6945e5ee56137d3d1489b63f071c1f23f
Successfully built aylien-news-api


## Fetch News with api

In [0]:
def fetch_new_stories(params={}):
	fetched_stories = []
	stories = None

	while stories is None or len(stories) > 0:
		response = api_instance.list_stories(**params)
		stories = response.stories
	
		params['cursor'] = response.next_page_cursor

		fetched_stories += stories
		#print("Fetched %d stories. Total story count so far: %d" %(len(stories), len(fetched_stories)))
	return fetched_stories

# Configure API key authorization: app_id
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = '######'
# Configure API key authorization: app_key
aylien_news_api.configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = '###########'

In [0]:
# create an instance of the API class
api_instance = aylien_news_api.DefaultApi()

params = {
	#'title': 'aapl OR apple OR AAPL OR Apple',
	#'text':'AAPL or aapl',
	'categories_confident': True,
	'categories_taxonomy': 'iptc-subjectcode',
	'categories_id': ['04000000'],
	'entities_body_links_dbpedia':[
    'http://dbpedia.org/resource/Apple_Inc.'
	],
	'source_rankings_alexa_rank_max': 150,
	'sort_by': 'source.rankings.alexa.rank',
	'sort_direction': 'asc',
	'language': ['en'],
	'published_at_start': 'NOW-10DAY',
	'published_at_end': 'NOW',
	#'cursor': '*',
	#'per_page': 10
}

stories = fetch_new_stories(params)

In [9]:
sentimentv = float()

for story in stories:
	print('Title: ' + story.title)
	print('Source: ' + story.source.name)
	print('Date: ' + str(story.published_at.date()))
	print('Title sentiment: ' + str(story.sentiment.title.score))
	print('Body sentiment: ' + str(story.sentiment.body.score))
	print('Link: ' + str(story.links.permalink) + '\n')
	sentimentv += story.sentiment.body.score

if (len(stories) > 0):
	sentimentv = sentimentv / len(stories)
	print('Average body sentiment: ' + str(sentimentv) + '\n')
	
print('************')
print("Fetched %d stories published between %s and %s" %(len(stories), params['published_at_start'], params['published_at_end']))

Title: Ariana Grande drops new album 'thank u, next' amid Grammys controversy
Source: Google News
Date: 2019-02-09
Title sentiment: 0.524799
Body sentiment: 0.990762
Link: https://www.goodmorningamerica.com/culture/story/ariana-grande-drops-album-amid-grammys-controversy-60936352

Title: Apple fixes FaceTime bug; will pay teen who found it
Source: Google News
Date: 2019-02-09
Title sentiment: 0.840944
Body sentiment: 0.664177
Link: https://www.syracuse.com/business/2019/02/apple-fixes-facetime-eavesdropping-bug-will-pay-teen-who-found-it.html

Title: Sprint Sues AT&T, Proving ‘5G’ Is Still Meaningless
Source: Google News
Date: 2019-02-09
Title sentiment: 0.39343
Body sentiment: 0.366289
Link: https://www.wired.com/story/sprint-sues-att-proving-5g-still-meaningless/

Title: This week’s best deals include $100 off the iPad and Xbox One X and big savings on the Amazon Echo
Source: Google News
Date: 2019-02-09
Title sentiment: 0.77564
Body sentiment: 0.890918
Link: https://www.theverge.com

## Newskeep

In [0]:
import pandas as pd
import numpy as np
from collections import deque

In [11]:
datax = deque()

for story in stories:
    x = list()
    x.append(story.title)
    x.append(story.source.name)
    x.append(story.published_at.date())
    x.append(story.sentiment.title.score)
    x.append(story.sentiment.body.score)
    x.append(' '.join(story.summary.sentences))
    print(story.summary.sentences)
    x.append(story.links.permalink)
    datax.append(x)
 
storage = np.array(datax)
datax = np.array(datax)

[]
['Apple fixed a FaceTime bug on Thursday that allowed people to eavesdrop and also said it will pay a 14-year-old high school student for reporting the problem.', 'The bug was related to Apple’s group FaceTime feature.', 'Grant Thompson, a freshman at Catalina Foothills High School in Tucson, Arizona, found the bug a little over two weeks ago during a chat with friends while playing the video game “Fortnite,” according to The Wall Street Journal.', 'Apple has a "bug bounty" program that in some cases can pay up to hundreds of thousands of dollars to researchers reporting bugs, according to the Journal.', 'Apple deemed the flaw so serious that it disabled the group FaceTime feature while it worked on a fix, according to the BBC.']
["But Apple doesn't sell an iPhone that supports 5G standards, and ATT doesn't yet offer 5G service for mobile phones.", 'ATT has branded parts of its 4G network as 5GE, or "5G Evolution."', '"5G Evolution and the 5GE indicator simply let customers know whe

In [12]:
headers = ['Title','Source','Date','Title Sentiment','Body Sentiment','Summary']
df = pd.DataFrame(data=datax[0:,0:],columns=['Title','Source','Date','Title Sentiment','Body Sentiment','Summary','Link'])
df.index += 1
df.to_csv('export.csv')

print('\n')
print('Collected Articles are written in export.csv')



Collected Articles are written in export.csv


## News Scrubbing

In [13]:
import pandas as pd
import numpy as np
import bokeh
#import vader

pd.options.mode.chained_assignment = None


df = pd.read_csv('export.csv')
df2=df[['Title','Date','Title Sentiment','Body Sentiment']]
df2['Weighted Sentiment'] = pd.Series(np.random.randn(len(df2['Title'])), index=df2.index)


for i in range(len(df2['Title'])):
    x = df2['Title Sentiment'][int(i)]
    y = df2['Body Sentiment'][int(i)]
    weight = (0.8*x) + (0.2*y)
    df2['Weighted Sentiment'][i] = weight

print(df2)    

                                                 Title        Date  \
0    Ariana Grande drops new album 'thank u, next' ...  2019-02-09   
1    Apple fixes FaceTime bug; will pay teen who fo...  2019-02-09   
2    Sprint Sues AT&T, Proving ‘5G’ Is Still Meanin...  2019-02-09   
3    This week’s best deals include $100 off the iP...  2019-02-09   
4    2017 vs 2018 MacBook Air - Real World Differen...  2019-02-09   
5    Apple Expands Availability of Refurbished 2018...  2019-02-09   
6    Apple being sued because two-factor authentica...  2019-02-09   
7    Apple finally fixes eavesdropping Group FaceTi...  2019-02-09   
8    LG G8 ThinQ price leaked – it could undercut t...  2019-02-11   
9    Uh oh: Insider says Samsung’s foldable Galaxy ...  2019-02-11   
10   Huawei sales soar as iPhones slump in China fo...  2019-02-11   
11   Apple iPhone Shipments Dive in China as Huawei...  2019-02-11   
12   Apple partners with VA to bring Health Records...  2019-02-11   
13                  